# MTSのZにSN比の効果ゲインを重みづけを行う

In [5]:
import pandas as pd
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.stats import chi2
import matplotlib.dates as mdates
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score


In [65]:
df = pd.read_csv('../data/letter_recognition.csv', header=None)

#Aのみを判定するため，Aを１，A以外を0にした．
df[0] = df[0].apply(lambda x: 0 if x == 'A' else 1)

#Xとyを入力
X = df[range(1,17)]
y = df[0]

#バギング側の話
#ブートストラップサンプリングの個数
n = 10
seed = random.randint(0, n)

#使用する7つの変数をランダムに取得する
random.seed(6)
random_s = random.sample(list(X.columns), 7)
use_X = X[random_s]

X_train, X_test, y_train, y_test = train_test_split(use_X, y, test_size=0.2)

In [66]:
# 必要な関数の定義

# 共分散行列の逆行列
def inv_cov(Z):
    #標準化後のベクトルを入力する
    #標準化した後なので相関行列と分散共分散行列は一致する
    c = np.cov(Z.T)
    return np.linalg.pinv(c)

#マハラノビス汎距離
def cal_MD(Z, inv_C):
    '''
    Z:標準化したベクトル
    inv_C:分散共分散行列の逆行列
    '''
    MD = np.zeros(len(Z))
    for i in range(len(Z)):
        _a = np.dot(Z[i], inv_C)
        _MD = np.dot(_a, Z[i].T)
        _MD = _MD / Z.shape[1]
        MD[i] = _MD
    return MD
    
# 閾値をジニ係数が最小になるように決定する
def determine_threshold(y_true, y_proba):
    """
    input: 
        y_true: trainデータのラベルを入力
        y_proba: trainデータの異常度（縮小モデルのMD）を入力
    output: threshold
    """
    df_ = pd.DataFrame(y_true)
    df_['proba'] = y_proba
    df_ = df_.sort_values('proba').reset_index(drop=True)

    min_gini = np.inf
    threshold = 0
    for i in range(len(df_)):
        
        neg = df_.iloc[:i+1]
        pos = df_.iloc[i:]

        p_neg = sum(neg[y_true.name]) / len(neg)
        gini_neg = 1 - ( p_neg ** 2 + ( 1 - p_neg ) ** 2 )

        p_pos = sum(pos[y_true.name]) / len(pos)
        gini_pos = 1 - ( p_pos ** 2 + ( 1 - p_pos ) ** 2 )

        gini_split = (len(neg) / len(df_) * gini_neg) + (len(pos) / len(df_) * gini_pos)

        if min_gini > gini_split:
            min_gini = gini_split
            threshold = df_.iloc[i]['proba']
            threshold_idx = i

    return threshold

## 重みづけなしのMTS

In [67]:
# MTSを実行
def fit_MTS(X, y):
    """
    input: X, y
    output: reduced_model_scaler, reduced_model_inv_C, select_columns

    reduced_model_scaler: 縮小モデルのスケーラー
    reduced_model_inv_C: 縮小モデルの共分散行列の逆行列
    select_columns: 選択された変数
    """
    # 正常データのみを使用して標準化
    scaler = StandardScaler()
    scaler.fit(X[y == 0])
    normal_Z = scaler.transform(X[y == 0])
    anomaly_Z = scaler.transform(X[y == 1])

    # 正常データのみを使用して共分散行列を計算
    inv_C = inv_cov(normal_Z)

    # いったん飛ばす，削除の基準は？削除しない方法もあるっぽい？
        #１度目の仮のマハラノビス距離を計算
        # MD_1st = cal_MD(normal_Z, inv_C)
        # もしもマハラノビス距離が余りにも大きいサンプルがあれば任意で削除する
        # 削除後のデータを使用して標準化と共分散行列を計算

    # 異常データと直交表を用いてSN比を計算
    #L8直行表
    l8 = np.array([
        [1,1,1,1,1,1,1],
        [1,1,1,2,2,2,2],
        [1,2,2,1,1,2,2],
        [1,2,2,2,2,1,1],
        [2,1,2,1,2,1,2],
        [2,1,2,2,1,2,1],
        [2,2,1,1,2,2,1],
        [2,2,1,2,1,1,2]
        ])
    l8 = (l8 == 1)

    # 異常データのマハラノビス距離
    anomaly_MD = np.zeros((l8.shape[0], anomaly_Z.shape[0]))
    for i, l8_row in enumerate(l8):
        anomaly_MD[i] = cal_MD(anomaly_Z[:, l8_row], inv_C[l8_row][:,l8_row]) # 正常データのinv_Cを使う必要がある

    # SN比の算出
    sn = np.zeros(l8.shape[0])
    for idx, row in enumerate(anomaly_MD):
        sum_MD = 0
        for row_i in row:
            sum_MD += 1 / row_i
        sn[idx] = -10 * math.log10(sum_MD / len(row))
        
    # SN比を利用し，不要と思われる変数を削除する
    # 変数選択
    df_gain = pd.DataFrame(index=X.columns, columns=['効果ゲイン','残す'])
    for i, clm in enumerate(X.columns):
        gain = sum(sn[l8.T[i]]) - sum(sn[~l8.T[i]])
        df_gain.loc[df_gain.index == clm, '効果ゲイン'] = gain
        df_gain.loc[df_gain.index == clm, '残す'] = gain > 0
    # 選択された変数を保存
    select_columns = df_gain[df_gain['残す']].index
    
    # 選択された変数が1つ以下の場合の例外処理
    if len(select_columns) > 1:
        # 縮小モデルでのスケーラーと共分散行列を計算
        reduced_model_scaler = StandardScaler()
        reduced_model_scaler.fit(X[select_columns][y == 0])
        reduced_model_normal_Z = reduced_model_scaler.transform(X[select_columns][y == 0])
        reduced_model_inv_C = inv_cov(reduced_model_normal_Z)
    # 選択された変数が一つ以下の場合はその変数を正常データの平均と標準偏差で標準化してそれの二乗を異常値とする
    else:
        select_columns = df_gain['効果ゲイン'].astype(float).idxmax()
        reduced_model_scaler = X[select_columns][y == 0].mean()
        reduced_model_inv_C = X[select_columns][y == 0].std()

    # 縮小モデルのスケーラーと共分散行列と選択した変数を出力
    return reduced_model_scaler, reduced_model_inv_C, select_columns
# 縮小モデルによってマハラノビス距離を計算する
def cal_MD_by_reduced_model(X, reduced_model_scaler, reduced_model_inv_C, select_columns):
    # select_columnsがfloatになることがある？
    if type(reduced_model_scaler) == StandardScaler:
        Z = reduced_model_scaler.transform(X[select_columns])
        MD = cal_MD(Z, reduced_model_inv_C)
    # 変数が一つしか選択されなかった場合はその変数を正常データの平均と標準偏差で標準化してそれの二乗を異常値とする
    else:
        MD = ((X[select_columns] - reduced_model_scaler) / reduced_model_inv_C) ** 2
    return MD
def predict_MTS(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns, threshold):
    proba = cal_MD_by_reduced_model(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns)
    pred = proba > threshold
    return proba, pred

## 重みづけMTS
1. 単位空間の作成
    1. Xを正常データのみで標準化してZを取得
    2. 正常データのみで共分散行列の逆行列Inv_Cを取得
2. 縮小単位空間作成のための変数選択
    1. 異常データ（anomaly_Z）を用いてSN比を算出
    2. 直交表(現状はL8のみ)を用いて各変数の効果ゲインを算出
    3. 効果ゲインが負の変数を削除し，縮小単位空間を作成
    4. <u>**縮小単位空間において効果ゲインを正規化したものをその変数の重みとして保存**</u>
3. 縮小単位空間の閾値決定
    1. gini係数が最小となる閾値を算出
4. 新しいデータの予測
    1. 縮小単位空間によって新しいデータのマハラノビス距離を算出し，異常度とする．
    2. その異常度が閾値を超えたら異常と予測する．

In [69]:
# MTSを実行
def fit_WMTS(X, y):
    """
    input: X, y
    output: reduced_model_scaler, reduced_model_inv_C, select_columns

    reduced_model_scaler: 縮小モデルのスケーラー
    reduced_model_inv_C: 縮小モデルの共分散行列の逆行列
    select_columns: 選択された変数
    """
    # 正常データのみを使用して標準化
    scaler = StandardScaler()
    scaler.fit(X[y == 0])
    normal_Z = scaler.transform(X[y == 0])
    anomaly_Z = scaler.transform(X[y == 1])

    # 正常データのみを使用して共分散行列を計算
    inv_C = inv_cov(normal_Z)

    # いったん飛ばす，削除の基準は？削除しない方法もあるっぽい？
        #１度目の仮のマハラノビス距離を計算
        # MD_1st = cal_MD(normal_Z, inv_C)
        # もしもマハラノビス距離が余りにも大きいサンプルがあれば任意で削除する
        # 削除後のデータを使用して標準化と共分散行列を計算

    # 異常データと直交表を用いてSN比を計算
    #L8直行表
    l8 = np.array([
        [1,1,1,1,1,1,1],
        [1,1,1,2,2,2,2],
        [1,2,2,1,1,2,2],
        [1,2,2,2,2,1,1],
        [2,1,2,1,2,1,2],
        [2,1,2,2,1,2,1],
        [2,2,1,1,2,2,1],
        [2,2,1,2,1,1,2]
        ])
    l8 = (l8 == 1)

    # 異常データのマハラノビス距離
    anomaly_MD = np.zeros((l8.shape[0], anomaly_Z.shape[0]))
    for i, l8_row in enumerate(l8):
        anomaly_MD[i] = cal_MD(anomaly_Z[:, l8_row], inv_C[l8_row][:,l8_row]) # 正常データのinv_Cを使う必要がある

    # SN比の算出
    sn = np.zeros(l8.shape[0])
    for idx, row in enumerate(anomaly_MD):
        sum_MD = 0
        for row_i in row:
            sum_MD += 1 / row_i
        sn[idx] = -10 * math.log10(sum_MD / len(row))
        
    # SN比を利用し，不要と思われる変数を削除する
    # 変数選択
    df_gain = pd.DataFrame(index=X.columns, columns=['効果ゲイン','残す'])
    for i, clm in enumerate(X.columns):
        gain = sum(sn[l8.T[i]]) - sum(sn[~l8.T[i]])
        df_gain.loc[df_gain.index == clm, '効果ゲイン'] = gain
        df_gain.loc[df_gain.index == clm, '残す'] = gain > 0
    # 選択された変数を保存
    select_columns = df_gain[df_gain['残す']].index
    select_gain = df_gain[df_gain['残す']]['効果ゲイン'].values
    select_columns_weight = select_gain / select_gain.sum()
    
    # 選択された変数が1つ以下の場合の例外処理
    if len(select_columns) > 1:
        # 縮小モデルでのスケーラーと共分散行列を計算
        reduced_model_scaler = StandardScaler()
        reduced_model_scaler.fit(X[select_columns][y == 0])
        reduced_model_normal_Z = reduced_model_scaler.transform(X[select_columns][y == 0])
        reduced_model_inv_C = inv_cov(reduced_model_normal_Z)
    # 選択された変数が一つ以下の場合はその変数を正常データの平均と標準偏差で標準化してそれの二乗を異常値とする
    else:
        select_columns = df_gain['効果ゲイン'].astype(float).idxmax()
        reduced_model_scaler = X[select_columns][y == 0].mean()
        reduced_model_inv_C = X[select_columns][y == 0].std()

    # 縮小モデルのスケーラーと共分散行列と選択した変数を出力
    return reduced_model_scaler, reduced_model_inv_C, select_columns, select_columns_weight
# 縮小モデルによってマハラノビス距離を計算する
def cal_WMD_by_reduced_model(X, reduced_model_scaler, reduced_model_inv_C, select_columns, select_columns_weight):
    # select_columnsがfloatになることがある？
    if type(reduced_model_scaler) == StandardScaler:
        Z = reduced_model_scaler.transform(X[select_columns])
        Weighted_Z = Z * select_columns_weight
        MD = cal_MD(Weighted_Z, reduced_model_inv_C)
    # 変数が一つしか選択されなかった場合はその変数を正常データの平均と標準偏差で標準化してそれの二乗を異常値とする
    else:
        MD = ((X[select_columns] - reduced_model_scaler) / reduced_model_inv_C) ** 2
    return MD
def predict_WMTS(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns, select_columns_weight, threshold):
    proba = cal_WMD_by_reduced_model(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns, select_columns_weight)
    pred = proba > threshold
    return proba, pred

In [72]:
df = pd.read_csv('../data/letter_recognition.csv', header=None)

#Aのみを判定するため，Aを１，A以外を0にした．
df[0] = df[0].apply(lambda x: 0 if x == 'A' else 1)

#Xとyを入力
X = df[range(1,17)]
y = df[0]

#バギング側の話
#ブートストラップサンプリングの個数
n = 10
seed = random.randint(0, n)

#使用する7つの変数をランダムに取得する
random.seed(7)
random_s = random.sample(list(X.columns), 7)
use_X = X[random_s]

X_train, X_test, y_train, y_test = train_test_split(use_X, y, test_size=0.2)

In [73]:
reduced_model_scaler, reduced_model_inv_C, select_columns, select_columns_weight = fit_WMTS(X_train, y_train)
y_proba_train = cal_WMD_by_reduced_model(X_train, reduced_model_scaler, reduced_model_inv_C, select_columns, select_columns_weight)
threshold = determine_threshold(y_train, y_proba_train)
proba, pred = predict_WMTS(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns, select_columns_weight, threshold)
print(roc_auc_score(y_test.values, proba))

0.9501359307359307


In [74]:
reduced_model_scaler, reduced_model_inv_C, select_columns = fit_MTS(X_train, y_train)
y_proba_train = cal_MD_by_reduced_model(X_train, reduced_model_scaler, reduced_model_inv_C, select_columns)
threshold = determine_threshold(y_train, y_proba_train)
proba, pred = predict_MTS(X_test, reduced_model_scaler, reduced_model_inv_C, select_columns, threshold)
print(roc_auc_score(y_test.values, proba))

0.9499021645021645


# スコアが良くなる時もある！！！

### 実験してみる